## Amostra de Solução

Fazendo os imports, para carregar um modelo de embedding do Hugging Face para português, e o dataset de treino.

In [ ]:
import yaml

import pandas as pd

from sentence_transformers import SentenceTransformer

confs = yaml.safe_load(open("confs.yaml"))
predictors = confs["predictors"] ### Importante! O cientista poderá usar apenas estas features para criar/aperfeiçoar o modelo
text_target = confs["text_target"]

c:\Users\darth\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Carregando uma amostra do dataset para mostrar o que é esperado do case em questão.

In [ ]:
SAMPLE = 2000 # Para mostrar o que é esperado

train = pd.read_parquet("dados/train.parquet").iloc[:SAMPLE]
X_train, y_train = train[predictors], train[text_target]

Carrega o [modelo de embeddings](https://huggingface.co/ricardo-filho/bert-base-portuguese-cased-nli-assin-2) Bi-Encoder (BERT) para o português.

In [ ]:
model = SentenceTransformer("ricardo-filho/bert-base-portuguese-cased-nli-assin-2")

c:\Users\darth\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
y_train_sentences = y_train.tolist()
y_train_embeddings = model.encode(
    y_train_sentences, show_progress_bar=True, device="cuda", batch_size=32
)

Batches: 100%|██████████| 63/63 [01:42<00:00,  1.63s/it]


In [7]:
X_train_sentences = X_train.apply(lambda x: " - ".join(x), axis=1).tolist()
X_train_embeddings = model.encode(
    X_train_sentences, show_progress_bar=True, device="cuda", batch_size=32
)

Batches: 100%|██████████| 63/63 [00:55<00:00,  1.13it/s]


In [11]:
X_train["prediction"] = y_train.iloc[
    model.similarity(X_train_embeddings, y_train_embeddings).argmax(axis=1)
].tolist()
X_train["target"] = y_train

C:\Users\darth\AppData\Local\Temp\ipykernel_452\1871977755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["prediction"] = y_train.iloc[
C:\Users\darth\AppData\Local\Temp\ipykernel_452\1871977755.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["target"] = y_train


### Performance

Computando a performance do modelo nesse pequeno Sample do dataset de treino

In [12]:
performance_df = X_train[["prediction", "target"]]
performance_df["correct"] = performance_df["prediction"] == performance_df["target"]

C:\Users\darth\AppData\Local\Temp\ipykernel_452\1044487981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_df["correct"] = performance_df["prediction"] == performance_df["target"]


In [14]:
top_1_precision = performance_df["correct"].mean()
print(f"Top 1 Precision: {top_1_precision:.2%}")

Top 1 Precision: 18.75%


### Importante!

Naturalmente é esperado a performance no dataset de treino completo e não apenas na amostra (2000 casos). O dataset de treino não será fornecido para o cientista, pois o será utilizado para avaliar a performance do modelo treinado (ou *fine-tunado*) para este case.

Para mais informações sobre o case, veja o README.md do repositório.